In [13]:
import numpy as np 
import pandas as pd
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing import text, sequence
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.callbacks import EarlyStopping, ModelCheckpoint
max_features = 20000
maxlen = 100
train = pd.read_csv("train.csv")
train = train.sample(frac=1)
list_sentences_train = train["comment_text"].fillna("").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes[0]].values

In [14]:
#Tokenize 

# from keras.utils.np_utils import to_categorical
# y = to_categorical(y, num_classes=None)

tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(
   list_tokenized_train, y, test_size=0.50, random_state=42)

In [16]:
#Padding
X_t = sequence.pad_sequences(X_train, maxlen=maxlen)
X_v = sequence.pad_sequences(X_valid, maxlen=maxlen)

In [17]:
from keras.models import Model
def get_model(embed_size):
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='mean_squared_error', optimizer='sgd',
                  metrics=['accuracy'])
    return model
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')

In [18]:
EMBEDDING_FILE= 'glove.6B.50d.txt'
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))
embed_size = 50
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, j in word_index.items():
    if j >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[j] = embedding_vector

In [19]:
#print(model.summary())
model = None
list_sentences_train
model = get_model(embed_size)

In [ ]:
#y_test
from tqdm import tqdm
emb_train = model.predict(X_t)
emb_valid = model.predict(X_v)
norm_train = []
norm_valid = []
"""
for i in tqdm(range(len(emb_train))):
    temp = []
    temp_2 = []
    for j in range(len(emb_train[i])):
        temp.append(np.linalg.norm(emb_train[i][j]))
        temp_2.append(np.linalg.norm(emb_valid[i][j]))
    norm_train.append(temp)
    norm_valid.append(temp_2)
"""
for i in tqdm(range(len(emb_train))):
    norm_train.append(np.ndarray.flatten(emb_train[i]))
    norm_valid.append(np.ndarray.flatten(emb_valid[i]))
from sklearn.linear_model import LogisticRegression
from sklearn import metrics as mtr
from sklearn.preprocessing import minmax_scale , normalize

lr = LogisticRegression()
lr.fit(norm_train,y_train)
#prob = lr.predict_proba(norm_valid)
prob = lr.predict(norm_valid)
print(mtr.roc_auc_score(y_valid, prob))

100%|██████████| 79785/79785 [00:13<00:00, 5720.36it/s] 


In [29]:
len(norm_train[0])

5000

In [22]:
prob
#print(mtr.roc_auc_score(y_valid, prob))

array([[0.62810803, 0.37189197],
       [0.45719746, 0.54280254],
       [0.45307746, 0.54692254],
       ...,
       [0.6265642 , 0.3734358 ],
       [0.44356738, 0.55643262],
       [0.44424747, 0.55575253]])

array([[-0.03543852,  0.9391847 ,  0.98451626, ..., -0.6024291 ,
         0.54854643, -1.0524588 ],
       [-0.03543852,  0.9391847 ,  0.98451626, ..., -0.6024291 ,
         0.54854643, -1.0524588 ],
       [-0.03543852,  0.9391847 ,  0.98451626, ..., -0.6024291 ,
         0.54854643, -1.0524588 ],
       ...,
       [-0.37085   , -0.072561  , -0.44149   , ...,  0.29768   ,
        -0.94829   ,  1.9092    ],
       [ 0.1591    , -0.21428   ,  0.63099   , ...,  0.021215  ,
        -0.14219   ,  0.66955   ],
       [-0.37085   , -0.072561  , -0.44149   , ...,  0.29768   ,
        -0.94829   ,  1.9092    ]], dtype=float32)